In [1]:
!pip install pyloudnorm pydub numpy

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 6.2 MB/s eta 0:00:00a 0:00:01


In [2]:
#峰值归一化，但英文写出响度hh离谱
from pydub import AudioSegment, effects
import os
def batch_loudness_normalize(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for file in os.listdir(input_dir):
        if file.endswith(".wav"):
            path = os.path.join(input_dir, file)
            audio = AudioSegment.from_wav(path)
            normalized = effects.normalize(audio, headroom=1.0)
            normalized.export(os.path.join(output_dir, file), format="wav")
    print(f"Loudness normalization done: {output_dir}")

# 使用
#[可配置]
batch_loudness_normalize("./pre_deal/enhancement/mix","./pre_deal/enhancement/mix_normalized")

#也许考虑将视频截断时长增加说不定能提高一点点

Loudness normalization done: ./pre_deal/enhancement/mix_normalized


In [3]:
#简化版本 - 当检测到削波警告时跳过响度归一化
import os
import numpy as np
import pyloudnorm as pyln
from pydub import AudioSegment, effects
import warnings
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def batch_normalize(input_dir, output_dir, target_lufs=-16.0, headroom_db=1.0):
    """
    简化版本 - 当检测到削波警告时跳过响度归一化
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for file in os.listdir(input_dir):
        if file.endswith(".wav"):
            input_path = os.path.join(input_dir, file)
            output_path = os.path.join(output_dir, file)
            
            logger.info(f"处理文件: {file}")
            
            try:
                audio = AudioSegment.from_wav(input_path)
                
                # 尝试响度归一化
                clipping_detected = False
                
                with warnings.catch_warnings(record=True) as w:
                    warnings.simplefilter("always")
                    
                    try:
                        samples = np.array(audio.get_array_of_samples()).astype(np.float32)
                        samples = samples / (2**(audio.sample_width * 8 - 1))
                        
                        meter = pyln.Meter(audio.frame_rate)
                        loudness = meter.integrated_loudness(
                            samples.reshape(-1, 1) if audio.channels == 1 else samples.reshape(-1, audio.channels))
                        
                        logger.info(f"  当前响度: {loudness:.2f} LUFS")
                        
                        # 尝试应用响度归一化
                        loudness_normalized = pyln.normalize.loudness(samples, loudness, target_lufs)
                        
                        # 检查是否有削波警告
                        if any("Possible clipped samples in output" in str(warning.message) for warning in w):
                            clipping_detected = True
                            logger.warning(f"  检测到削波警告，跳过响度归一化")
                    except Exception as e:
                        logger.error(f"  响度归一化失败: {str(e)}")
                        clipping_detected = True
                
                # 根据是否检测到削波决定处理方式
                if clipping_detected:
                    # 只进行峰值归一化
                    peak_normalized = effects.normalize(audio, headroom=headroom_db)
                else:
                    # 进行完整的响度和峰值归一化
                    loudness_normalized = (loudness_normalized * (2**15 - 1)).astype(np.int16)
                    normalized_audio = AudioSegment(
                        loudness_normalized.tobytes(),
                        frame_rate=audio.frame_rate,
                        sample_width=audio.sample_width,
                        channels=audio.channels
                    )
                    peak_normalized = effects.normalize(normalized_audio, headroom=headroom_db)
                
                # 导出文件
                peak_normalized.export(output_path, format="wav")
                logger.info(f"  已处理并保存: {file}")
                
            except Exception as e:
                logger.error(f"  处理文件 {file} 时出错: {str(e)}")
                # 如果出错，尝试仅使用峰值归一化
                try:
                    audio = AudioSegment.from_wav(input_path)
                    peak_normalized = effects.normalize(audio, headroom=headroom_db)
                    peak_normalized.export(output_path, format="wav")
                    logger.info(f"  已使用峰值归一化处理: {file}")
                except Exception as e2:
                    logger.error(f"  峰值归一化也失败: {str(e2)}")




#[可配置]
batch_normalize("./pre_deal/enhancement/mix","./pre_deal/enhancement/mix_dnor")
print("?!")

INFO:__main__:处理文件: reference_1.wav
INFO:__main__:  当前响度: -46.58 LUFS
INFO:__main__:  已处理并保存: reference_1.wav
INFO:__main__:处理文件: reference_10.wav
INFO:__main__:  当前响度: -31.68 LUFS
INFO:__main__:  已处理并保存: reference_10.wav
INFO:__main__:处理文件: reference_100.wav
INFO:__main__:  当前响度: -38.51 LUFS
INFO:__main__:  已处理并保存: reference_100.wav
INFO:__main__:处理文件: reference_101.wav
INFO:__main__:  当前响度: -39.94 LUFS
INFO:__main__:  已处理并保存: reference_101.wav
INFO:__main__:处理文件: reference_102.wav
INFO:__main__:  当前响度: -22.70 LUFS
INFO:__main__:  已处理并保存: reference_102.wav
INFO:__main__:处理文件: reference_103.wav
INFO:__main__:  当前响度: -16.40 LUFS
INFO:__main__:  已处理并保存: reference_103.wav
INFO:__main__:处理文件: reference_104.wav
INFO:__main__:  当前响度: -37.80 LUFS
INFO:__main__:  已处理并保存: reference_104.wav
INFO:__main__:处理文件: reference_105.wav
INFO:__main__:  当前响度: -26.54 LUFS
INFO:__main__:  已处理并保存: reference_105.wav
INFO:__main__:处理文件: reference_106.wav
INFO:__main__:  当前响度: -26.84 LUFS
INFO:__main__:  已处理并保存

?!
